In [2]:
import pandas as pd

# Read CSV file into another DataFrame
df_csv = pd.read_csv('test_hate_dataset.csv', encoding='utf-8')
df_csv.head()
# Remove the single quote at the beginning of each sentence in the 'text' column
df_csv['text'] = df_csv['text'].str.replace("^'", "", regex=True)
#df_csv.iloc[4]['text']
df_csv['text'] = df_csv['text'].str.strip()
len(df_csv)

16458

In [5]:
import pandas as pd
import json
import os

# Define the root folder containing the JSON files
root_folder_path = 'extraction_05'

# List to store individual DataFrames
dataframes = []
count=0
# Recursively walk through the folder structure
for dirpath, dirnames, filenames in os.walk(root_folder_path):

    for filename in filenames:
        if filename.endswith('.json'):
            file_path = os.path.join(dirpath, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # Transform JSON to DataFrame and append to the list
                    df = pd.DataFrame(data)
                    dataframes.append(df)
            except json.JSONDecodeError as e:
                count+=1
                #print(f"Error decoding JSON file {file_path}: {e}")
            except Exception as e:
                count+=1

                #print(f"Unexpected error with file {file_path}: {e}")
print("Corrupted files",count, "missing sentences are much more!!!!")
# Concatenate all DataFrames if any valid data was found
if dataframes:
    final_df = pd.concat(dataframes, ignore_index=True)
    print(final_df)
else:
    print("No valid JSON files found.")


Corrupted files 367 missing sentences are much more!!!!
                                           full_sentence  sentence_number  \
0      social services have a love/hate relationship ...                0   
1      Yes it's a joke and offensive isn't it? Why ar...                1   
2      I do not have passions in life, and I'm not su...                2   
3      You better be. That was the most half-assed co...                3   
4      Waging war on, and bombing, a foreign sovereig...                4   
...                                                  ...              ...   
14621  I am getting more knowledgable in Islam thanks...                2   
14622  go fuck yourself if you ride camels in dubai w...                3   
14623                           All foreigners are dirty                4   
14624  Seeing my dumb ass brothers learning to swim i...                0   
14625  My fucking wish is to have enough money to mak...                1   

                   

In [7]:

final_df = final_df.rename(columns={'full_sentence': 'text'})

""" final_df['text']= emoji.replace_emoji(final_df['text'])
df_csv['text']= emoji.replace_emoji(df_csv['text'])
 """
#remove the ' or " at the beginning and ending of the string for both the dataframes
final_df['text'] = final_df['text'].str.replace("^'", "", regex=True)
final_df['text'] = final_df['text'].str.replace("'$", "", regex=True)
final_df['text'] = final_df['text'].str.replace("^\"", "", regex=True)
final_df['text'] = final_df['text'].str.replace("\"$", "", regex=True)
df_csv['text'] = df_csv['text'].str.replace("^'", "", regex=True)
df_csv['text'] = df_csv['text'].str.replace("'$", "", regex=True)
df_csv['text'] = df_csv['text'].str.replace("^\"", "", regex=True)
df_csv['text'] = df_csv['text'].str.replace("\"$", "", regex=True)

#replace the " with ' 
df_csv['text'] = df_csv['text'].str.replace("\"", "'", regex=True)
final_df['text'] = final_df['text'].str.replace("\"", "'", regex=True)

#remove extra blank spaces 
df_csv['text'] = df_csv['text'].str.strip()
final_df['text'] = final_df['text'].str.strip()

merged_df = pd.merge(final_df, df_csv, on='text', how='inner')

#print(merged_df)
print("Length json",len(final_df))

print("Length merged",len(merged_df))
print(df_csv.iloc[11465]['text'])

left_out_senteces= pd.merge(final_df, merged_df, on ='text', how='left')
print()
print(left_out_senteces[left_out_senteces['label'].isnull()].iloc[9]['text'])

Length json 14626
Length merged 12914
36 and have never bothered with women because they're a w*ste of time and space. Now, I'm ahead of the game.

Dear foreigners, stop saying you have pure intentions when you clearly dont


In [8]:
merged_df.head()

,text,sentence_number,topics,label
0,social services have a love/hate relationship ...,0,"[{'sentence_slice': 'social services', 'topic'...",nothate
1,Yes it's a joke and offensive isn't it? Why ar...,1,"[{'sentence_slice': 'trans men', 'topic': 'Gen...",nothate
2,"I do not have passions in life, and I'm not su...",2,"[{'sentence_slice': 'passions in life', 'topic...",nothate
3,You better be. That was the most half-assed co...,3,"[{'sentence_slice': 'comeback attempt', 'topic...",nothate
4,"Waging war on, and bombing, a foreign sovereig...",4,"[{'sentence_slice': 'war', 'topic': 'Conflict'...",hate


In [9]:
type(merged_df.iloc[0]['topics'][0])

dict

In [12]:
merged_df.to_pickle("merged_df.pk")

In [13]:
merged_df.to_csv("merged_df.csv")